In [1]:
using Pkg, Revise
Pkg.activate("../GenLinAlgProblems")

using GenLinAlgProblems, LinearAlgebra, ToeplitzMatrices, DSP, Polynomials, SymPy, Latexify, LaTeXStrings, Random, StatsBase, IJulia
using Plots, Printf

x = symbols("x", real=true)
z = [symbols("z_$i", real=true) for i in 1:7]
c = [symbols("c_$i", real=true) for i in 0:4]
Random.seed!( 12345 );

  Activating project at `C:\Users\jeff\NOTEBOOKS\elementary-linear-algebra\GenLinAlgProblems`


<div style="float:center;width:100%;text-align: center;">
    <strong style="height:60px;color:darkred;font-size:40px;">Circulant Matrices</strong><br>
</div>

**References**
* [**I. Kra, S. R. Simanca, "On Circulant Matrices"**](https://www.ams.org/journals/notices/201203/rtx120300368p.pdf)
* [<strong>https://web.mit.edu/18.06/www/Spring17/Circulant-Matrices.pdf</strong>](https://web.mit.edu/18.06/www/Spring17/Circulant-Matrices.pdf)

# 1. Non-cyclic (Linear) and Cyclic Convolution

## 1.1 Linear (non-cyclic) Convolution

### 1.1.1 Definition

<div style="background-color:#F2F5A9;color:black;">

**Definition**: Given two vectors $f$ and $g$ of lengths $M$ and $N$ respectively,<br>
$\qquad$ their **linear convolution** is a vector of length $M+N-1$.<br><br>
$\qquad\qquad (f * g)_n = \sum_{m=\max(0,n-N+1)}^{\min(M-1,n)}\; f_m\; g_{n-m}$<br><br>
$\qquad$ Note that the result is longer than both input vectors<br>
</div>

**Polynomial interpretation:** Linear convolution corresponds to **polynomial multiplication** in the ring of polynomials $F[X]$.

### 1.1.2 Examples

* **Example:** $f = [1, 2, 3]$ and $g = [4, 5],\;\;$ then $f*g = [4, 13, 22, 15]$<br><br>
<div style="float:left;padding-left:1cm;width:40%;">
<table style="border-collapse: collapse width: 30%;float:left;">
  <tr style="border: 1px solid black;">
    <td>f</td>
    <td><span style="display: inline-block; width: 3cm;">&nbsp;</span></td>
    <td></td>
    <td style="text-align: center;">1</td>
    <td style="text-align: center;">2</td>
    <td style="text-align: center;">3</td>
  </tr>
  <tr style="border: 1px solid black;border-bottom: 2px solid black;">
    <td>reversed g</td>
    <td></td>
    <td></td>
    <td></td>
    <td style="text-align: center;">4</td>
    <td style="text-align: center;">5</td>
  </tr>
  <tr style="border: 1px solid black;">
    <td>product f×5 </td>
    <td></td>
    <td></td>
    <td style="text-align: center;">5</td>
    <td style="text-align: center;">10</td>
    <td style="text-align: center;">15</td>
  </tr>
  <tr style="border: 1px solid black;border-bottom: 2px solid black;">
    <td>product f×4</td>
    <td></td>
    <td style="text-align: center;">4</td>
    <td style="text-align: center;">8</td>
    <td style="text-align: center;">12</td>
    <td></td>
  </tr>
  <tr style="border: 1px solid black;">
    <td>sum of products</td>
    <td></td>
    <td style="text-align: center;">4</td>
    <td style="text-align: center;">13</td>
    <td style="text-align: center;">22</td>
    <td style="text-align: center;">15</td>
  </tr>
</table>
</div><div style="float:left;padding-left:1cm;width:40%;">
The linear convolution can be represented in the same way as<br> multiplication of numbers (without carry)<br><br>
<strong>Remark:</strong> A check of the result is $\sum{f}\ \sum{g} = \sum{f*g},\;$<br>
$\qquad\qquad ( 1+2+3 ) (4+5) = ( 4+13+22 +15 )$
</div>

* **Example:** Given $\;\;f = 1+2x+3 x^2\;\;$ and $\;\;g = 4 + 5x,\;\;$ then $\;\;f*g = 4 + 13 x+22 x^2 + 15 x^3$

In [2]:
@show conv( [1 2 3], [4 5]) == [4 13 22 15];
@show (1+2x+3x^2)*(4+5x) == (4+13x+22x^2+15x^3);

conv([1 2 3], [4 5]) == [4 13 22 15] = true
(1 + 2x + 3 * x ^ 2) * (4 + 5x) == 4 + 13x + 22 * x ^ 2 + 15 * x ^ 3 = true


### 1.1.3 Matrix representation

Linear convolution $f * g$ can be represented as a matrix-vector multiplication<br>
$\qquad$ $f*g = T f,\;\;$ where $T$ is a Toeplitz matrix formed from the vector $g$.

**Definition:** A **Toeplitz matrix** or diagonal-constant matrix, is a matrix in which each descending diagonal from left to right is constant.<br>
$\qquad$ (Note that a Toeplitz Matrix need not be square.)

$\qquad T = \begin{pmatrix} \color{red}{a_0} & a_{-1} & a_{-2} & \dots & a_{-n+1} \\
                            a_{1} & \color{red}{a_{0}} & a_{-1} & \dots & a_{-n+2} \\
                            a_{2} & a_{1} &  \color{red}{\ddots} & \ddots &  \ddots  \\
                            \ddots & \ddots &  \ddots & \color{red}{\ddots} &  \ddots \\
                            a_{n-1} & a_{n-2} & \dots & \dots & \color{red}{a_0} \end{pmatrix}\qquad$ (here only the main diagonal is shown in red)

To encode a vector $g$ of length $M$ in a Toeplitz matrix for computing the linear convolution with another vector $f$ of length $N$:<br>
 (N+M-1) × N.
Construct a Toeplitz matrix $T$ of size $(N+M-1) \times N$:
* The first column of $T$ contains the vector $g$ padded with $N-1$ zeros at the end.
* Each subsequent column is a downward shift of the previous column.

**Example:** Let $g = [4, 5, 6]$, and let $f$ have length 3, e.g., $f=[1,2,3]$ Then<br><br>
$\qquad T f = \begin{pmatrix}
4 & 0 & 0 \\
5 & 4 & 0 \\
6 & 5 & 4 \\
0 & 6 & 5 \\
0 & 0 & 6 \\
\end{pmatrix} \begin{pmatrix} 1 \\ 2 \\ 3 \end{pmatrix} = \left(
\begin{array}{r}
4 \\
13 \\
28 \\
27 \\
18 \\
\end{array}
\right)$

In [3]:
function convolution_matrix(g::Vector{T}, n::Int) where T
    g_padded = [g; zeros(T, n-1)]
    Toeplitz(g_padded, [g[1]; zeros(T, n-1)])
end
f = [1; 2; 3]
g = [4,5,6]
T = convolution_matrix(g, length(f))
println("f*g = ", conv(f,g))
@show T*f == conv(f,g);

f*g = [4, 13, 28, 27, 18]
T * f == conv(f, g) = true


## 1.2 Cyclic or Circular Convolution

### 1.2.1 Definition

<div style="background-color:#F2F5A9;color:black;">

**Definition:** Given two vectors $f$ and $g$ both of length $N$,<br>
$\qquad$ their cyclic or **circular convolution** is a vector of length $N$<br><br>
$\qquad\qquad (f \circledast g)_n = \sum_{m=0}^{N-1}\; f_m \; h_{n-m} \bmod N,\qquad$ where $\bmod$ is the modulo operation applied to the indices
</div>

The circular convolution has the following key properties:
<div style="float:left;padding-left:1.5cm;">

| Property | Formula |
| :-------- | :------- |
| **Periodicity:** | $\qquad (f \circledast g)_n = (f \circledast g)_{n+N}$ |
| **Commutativity:** | $\qquad f \circledast g = g \circledast f$ |
| **Associativity:** | $\qquad (g \circledast g) \circledast h = f \circledast (g \circledast h)$ |
| **Distributivity over addition:** | $\qquad f \circledast (g + h) = (f \circledast g) + (f \circledast h)$ |
</div>

### 1.2.2 Examples

#### Example: Vectors

In [4]:
function circular_conv(x::Vector{T}, h::Vector{T}) where T
    # naive implementation
    N = length(x)
    y = zeros(T, N)
    for n in 0:N-1
        for m in 0:N-1
            y[n+1] += x[m+1] * h[mod(n-m, N) + 1]
        end
    end
    return y
end
f = [1, 2, 3, 4]
g = [5, 6, 7, 8]
l_show(L"f =", f, L"\; g =", g, L"\qquad f \circledast g =", circular_conv(f,g))

L"$f =$$\begin{equation}
\left(
\begin{array}{r}
1 \\
2 \\
3 \\
4 \\
\end{array}
\right)
\end{equation}
$$\; g =$$\begin{equation}
\left(
\begin{array}{r}
5 \\
6 \\
7 \\
8 \\
\end{array}
\right)
\end{equation}
$$\qquad f \circledast g =$$\begin{equation}
\left(
\begin{array}{r}
66 \\
68 \\
66 \\
60 \\
\end{array}
\right)
\end{equation}
$"

The circular convolution wraps entries

In [5]:
function linear_to_circular_conv( linear_result::Vector{T}, N::Int ) where T
    circular_result = zeros(T, N)
    for i in 1:length(linear_result)
        circular_result[mod1(i, N)] += linear_result[i]  # wrap the linear convolution result
    end
    circular_result
end
l_show(  L"\qquad f \circledast g =", linear_to_circular_conv(conv(f,g), length(f) ))

L"$\qquad f \circledast g =$$\begin{equation}
\left(
\begin{array}{r}
66 \\
68 \\
66 \\
60 \\
\end{array}
\right)
\end{equation}
$"

#### Example: Polynomials of a Circular Shift Matrix

The basic circular shift matrix $P$ of size $N \times N$ applied to a vector cyclically shifts the entries of the vector.

In [6]:
function circular_shift_matrix(N::Int)
    C = zeros(Int, N, N)
    for i in 1:N-1   C[i, i+1] = 1 end
    C[N, 1] = 1
    C
end
function circular_shift(vector::Vector)
    [vector[2:end]; vector[1]] # Shift the vector up by one position circularly
end

v = z[1:5] #[1,2,3,4,5]
P = circular_shift_matrix(length(v))
display(l_show( "circular shift matrix P = ", P, L"\quad v = ", v, L"\qquad Pv =", P*v, L"\qquad P^2v =", P^2*v))
@show P*v == circular_shift(v);

"\$\\text{circular shift matrix P = }\$\$\\begin{equation}\n\\left(\n\\begin{array}{rrrrr}\n0 & 1 & 0 & 0 & 0 \\\\\n0 & 0 & 1 & 0 & 0 \\\\\n0 & 0 & 0 & 1 & 0 \\\\\n0 & 0 & 0 & 0 & 1 \\\\\n1 & 0 & 0 & 0 & 0 \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$\\quad v = \$\$\\begin{equation}\n\\left(\n\\begin{array" ⋯ 89 bytes ⋯ "quad Pv =\$\$\\begin{equation}\n\\left(\n\\begin{array}{r}\nz_{2} \\\\\nz_{3} \\\\\nz_{4} \\\\\nz_{5} \\\\\nz_{1} \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$\\qquad P^2v =\$\$\\begin{equation}\n\\left(\n\\begin{array}{r}\nz_{3} \\\\\nz_{4} \\\\\nz_{5} \\\\\nz_{1} \\\\\nz_{2} \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$"

P * v == circular_shift(v) = true


____
Repeated application of $P$ shifts the vector further.

In [7]:
l_show(L"z =", v, L",\quad P z =", P*v,  L",\quad P^2 z =", P^2*v,
       L",\quad P^3 z =", P^3*v,  L",\quad P^4 z =", P^4*v, L",\quad P^5 z =", P^5*v)

"\$z =\$\$\\begin{equation}\n\\left(\n\\begin{array}{r}\nz_{1} \\\\\nz_{2} \\\\\nz_{3} \\\\\nz_{4} \\\\\nz_{5} \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$,\\quad P z =\$\$\\begin{equation}\n\\left(\n\\begin{array}{r}\nz_{2} \\\\\nz_{3} \\\\\nz_{4} \\\\\nz_{5} \\\\\nz_{1} \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$,\\qua" ⋯ 280 bytes ⋯ "P^4 z =\$\$\\begin{equation}\n\\left(\n\\begin{array}{r}\nz_{5} \\\\\nz_{1} \\\\\nz_{2} \\\\\nz_{3} \\\\\nz_{4} \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$,\\quad P^5 z =\$\$\\begin{equation}\n\\left(\n\\begin{array}{r}\nz_{1} \\\\\nz_{2} \\\\\nz_{3} \\\\\nz_{4} \\\\\nz_{5} \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$"

Given a vector of length $N$, repeated application of the shift matrix $P$ returns the vector to its original order in $N$ steps.

In [8]:
l_show(L"\quad P =", P,  L"\quad P^2 =", P^2,  L"\quad P^3 =", P^3,  L"\quad P^4 =", P^4, L"\quad P^5 =", P^5)

"\$\\quad P =\$\$\\begin{equation}\n\\left(\n\\begin{array}{rrrrr}\n0 & 1 & 0 & 0 & 0 \\\\\n0 & 0 & 1 & 0 & 0 \\\\\n0 & 0 & 0 & 1 & 0 \\\\\n0 & 0 & 0 & 0 & 1 \\\\\n1 & 0 & 0 & 0 & 0 \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$\\quad P^2 =\$\$\\begin{equation}\n\\left(\n\\begin{array}{rrrrr}\n0 & 0 & 1 & 0 " ⋯ 457 bytes ⋯ "1 & 0 & 0 \\\\\n0 & 0 & 0 & 1 & 0 \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$\$\\quad P^5 =\$\$\\begin{equation}\n\\left(\n\\begin{array}{rrrrr}\n1 & 0 & 0 & 0 & 0 \\\\\n0 & 1 & 0 & 0 & 0 \\\\\n0 & 0 & 1 & 0 & 0 \\\\\n0 & 0 & 0 & 1 & 0 \\\\\n0 & 0 & 0 & 0 & 1 \\\\\n\\end{array}\n\\right)\n\\end{equation}\n\$"

Since  $P^N = I$ for a basic shift matrix of size $N\times N$, the multiplication of two polynomials in $P$<br>
can be expressed as a circular convolution of their respective coefficient vectors of size $N$. For example,

$\quad\left. \begin{align}
f(P) &= 3 I +2 P + P^3 \\
g(P) &= 2 P^3 \\
\end{align} \right\}\;\; \Rightarrow\;\;
\begin{pmatrix} 3\\ 2\\ 0\\ 1\\ 0\end{pmatrix} \circledast \begin{pmatrix}0\\ 0\\ 0\\ 2\\ 0 \end{pmatrix}
= \begin{pmatrix}0\\ 2\\ 0\\ 6\\ 4\end{pmatrix}\;\; \Rightarrow\;\;  f(P)\ g(P) = 2P + 6P^3+4 P^4$

In [9]:
@show circular_conv( [3, 2, 0, 1, 0], [0, 0, 0, 2, 0 ]) == [0,2,0,6,4];

circular_conv([3, 2, 0, 1, 0], [0, 0, 0, 2, 0]) == [0, 2, 0, 6, 4] = true


### 1.2.3 Matrix Representation

Circular convolution of two vectors $f$ and $g$ of length $N$ can be represented as a matrix-vector multiplication<br>
$\qquad$ $f \circledast g = T f,\;\;$ where $T$ is a Toeplitz matrix of size $N \times N$ formed from the vector $g$
* The first column of $T$ contains the vector $g$.
* Each subsequent column is a circular down shift of the previous column.

This type of matrix is known as a **Circulant Matrix**

In [10]:
function circular_convolution_matrix(v::Vector{T}) where T
    n = length(v)
    return [v[mod1(j - i + 1, n)] for j in 1:n, i in 1:n]
end

f = [1, 2, 3, 4]
g = [5, 6, 7, 8]
T = circular_convolution_matrix( g )
println("Reusing the same vectors f and g as before, we find")
l_show( L"\qquad f =", f, L",\quad g =", g, L", \quad T =", T, L"\qquad f \circledast g = T f = ", T*f )

Reusing the same vectors f and g as before, we find


L"$\qquad f =$$\begin{equation}
\left(
\begin{array}{r}
1 \\
2 \\
3 \\
4 \\
\end{array}
\right)
\end{equation}
$$,\quad g =$$\begin{equation}
\left(
\begin{array}{r}
5 \\
6 \\
7 \\
8 \\
\end{array}
\right)
\end{equation}
$$, \quad T =$$\begin{equation}
\left(
\begin{array}{rrrr}
5 & 8 & 7 & 6 \\
6 & 5 & 8 & 7 \\
7 & 6 & 5 & 8 \\
8 & 7 & 6 & 5 \\
\end{array}
\right)
\end{equation}
$$\qquad f \circledast g = T f = $$\begin{equation}
\left(
\begin{array}{r}
66 \\
68 \\
66 \\
60 \\
\end{array}
\right)
\end{equation}
$"

**Remarks:**
* The first row is the vector $g$ reversed
* Given the first row of $T$, each subsequent row is a circular right shift of the previous row
* Each diagonal of $T$ is constant and wraps from the $i^{th}$ diagonal to the $N-i^{th}-1$ diagonal,<br>
for a total number of $N$ entries each.

# 2. Circulant Matrices

## 2.1 Definition and Basic Properties

### 2.1.1 Definition

<div style="background-color:#F2F5A9;color:black;">

**Definition:** The circulant matrix $C = \text{circ}(v)$ associated to the vector $c \in \mathbb{C}^n$<br>
$\qquad$ is the $n\times n$ matrix whose rows are given by iterations of the circular shift Matrix $P$:<br>
$\qquad\qquad C =
\left(
\begin{array}{rrrrrr}
c_{0}  & c_{1}  & \dots  & c_{n-2} & c_{n-1} \\
c_{n-1}  & c_{0}  & \dots  & c_{n-3} & c_{n-2} \\
\vdots & \vdots & \ddots & \vdots  & \vdots \\
c_{2}  & c_{3}  & \dots  & c_{0}  & c_{1} \\
c_{1}  & c_{2}  & \dots  & c_{n-1}  & c_{0} \\
\end{array}
\right)
$
<br><br>
</div>

**Remark**: It will prove convenient to start the matrix and vector indices at 0 rather than 1.

In [11]:
function circulant_matrix(v::Vector{T}) where T
    n = length(v)
    return [v[mod1(j - i + 1, n)] for i in 1:n, j in 1:n]
end
C = circulant_matrix( c )
l_show( "Circulant Matrix ", L"\;\;C = ", C)

L"$\text{Circulant Matrix }$$\;\;C = $$\begin{equation}
\left(
\begin{array}{rrrrr}
c_{0} & c_{1} & c_{2} & c_{3} & c_{4} \\
c_{4} & c_{0} & c_{1} & c_{2} & c_{3} \\
c_{3} & c_{4} & c_{0} & c_{1} & c_{2} \\
c_{2} & c_{3} & c_{4} & c_{0} & c_{1} \\
c_{1} & c_{2} & c_{3} & c_{4} & c_{0} \\
\end{array}
\right)
\end{equation}
$"

A circulant matrix can be expressed as a polynomial of the circular shift matrix introduced in 1.2.2 above:

$\qquad C = c_0 I + c_1 P + c_2 P^2 + \dots + P^{n-1}$

In [12]:
P = circular_shift_matrix(length(c))
@show C == sum( [c[i]*P^(i-1) for i in 1:length(c)]);

C == sum([c[i] * P ^ (i - 1) for i = 1:length(c)]) = true


### 2.1.2 The Subspace of Circulant Matrices of Fixed Size

Let $\mathscr{C}_n$ be the set of circulant matrices of size $n \times n$.

Given the matrix polynomial representation $\;\; C = c_0 I + c_1 P + c_2 P^2 + \dots + P^{n-1}$ of a circular matrix, we trivially have
* $\mathscr{C}_n$ is **closed under addition and scalar multiplication** (and hence a subspace of the set of matrices $\mathscr{M}_n$.
* Given two circulant matrices $C_1$ and $C_2$
    * The product $C_1 C_2 \in \mathscr{C}_n$ is a circulant matrix:  $\mathscr{C}_n$ is **closed under matrix multiplication.**
    * The **product of circulant matrices is commutative:** $C_1 C_2 = C_2 C_1$

### 2.1.3 Eigenvalues and Eigenvectors

#### Eigenvalues and Eigenvectors of the Circular Shift Matrix $P$ of Size $n\times n$

* The matrix $P$ is normal: $P P^t = P^t P$. It therefore must have a set of $n$ orthonormal eigenvectors.<br>
Since $P$ is not symmetric, the eigenvalues will be complex.
* Since the rows of $P$ sum to 1, we trivially see that $\lambda=1$ is an eigenvalue for the eigenvector $x=\begin{pmatrix} 1\\1\\ \vdots \\ 1 \end{pmatrix}$
* In fact, the characteristic polynomial for $P$ is $\;\;p(\lambda) = (\lambda-1)^n$.<br>
Its roots are the roots on unity $\left\{ 1, \omega, \omega^2, \dots \omega^{n-1}\right\},\;$ where $\; \omega=e^{\frac{2\pi}{n} i}$.

* An eigenvector for eigenvalue $\omega^k$ is given by $x_k = \begin{pmatrix}
\omega^{0k}\\
\omega^{1k}\\
\omega^{2k}\\
\dots\\
\omega^{(n-1)k}\\
\end{pmatrix}$

* The resulting matrix of eigenvectors $F_n = \frac{1}{\sqrt{n}} \begin{pmatrix} x_0 & x_1 & \dots & x_{n-1}\end{pmatrix}$ is a [**Fourier Matrix**](FourierMatrix.ipynb).<br>
The factor by $\frac{1}{\sqrt{n}}$ scales the eigenvectors to unit length.

In [25]:
function plot_roots_of_unity_with_vectors(n::Int)
    roots           = [exp(2im * π * k / n) for k in 0:n-1]      # Calculate the n-th roots of unity
    real_parts      = real.(roots)
    imaginary_parts = imag.(roots)

    # Define and plot the unit circle
    θ = range(0, 2π, length=100)
    unit_circle_x = cos.(θ)
    unit_circle_y = sin.(θ)

    p = plot(unit_circle_x, unit_circle_y, label="Unit Circle", aspect_ratio=:equal, color=:black,
         xlabel="Real Part", ylabel="Imaginary Part", title="$n-th Roots of Unity",
         legend=:outertopright, grid=true, size=(320,320))

    scatter!(real_parts, imaginary_parts, label="Roots", color="#989F7A")  # add points for each root
    quiver!(zeros(n), zeros(n), quiver=(real_parts, imaginary_parts),  # plot the vectors to the roots
            arrow=true, label="Vectors", color="#60708B")

    table_data = [(k-1, real(root), imag(root)) for (k, root) in enumerate(roots)]
    return p, table_data

end
# ================================================================================================================
function display_roots_of_unity_html(n::Int, plot_obj, data)
    plot_svg = sprint(show, MIME("image/svg+xml"), plot_obj)

    table_html = """
    <table style="border-collapse: collapse; font-size: 0.9em; width: auto;">
        <tr style="background-color: #f2f2f2;">
            <th style="border: 1px solid #ddd; padding: 4px 8px;">k</th>
            <th style="border: 1px solid #ddd; padding: 4px 8px;">Real</th>
            <th style="border: 1px solid #ddd; padding: 4px 8px;">Imaginary</th>
        </tr>
        $(join(["<tr><td style='border: 1px solid #ddd; padding: 4px 8px; text-align: center;'>$k</td><td style='border: 1px solid #ddd; padding: 4px 8px; text-align: right;'>$(@sprintf("%.4f", re))</td><td style='border: 1px solid #ddd; padding: 4px 8px; text-align: right;'>$(@sprintf("%.4f", im))</td></tr>" for (k, re, im) in data]))
    </table>
    """
    html_output = """
    <div style="display: flex; justify-content: flex-start; align-items: flex-start; gap: 2cm;">
        <div style="flex: none;">
            $plot_svg
        </div>
        <div style="flex: none;">
            <h3 style="font-size: 1em; margin-bottom: 5px;margin-top:1.5cm;">$n-th Roots of Unity:</h3>
            $table_html
        </div>
    </div>
    """
    display("text/html", html_output)
end

# ================================================================================================================
function show_roots_of_unity(n::Int)
    plot_obj, data = plot_roots_of_unity_with_vectors(n)
    display_roots_of_unity_html(n, plot_obj, data)
end
# ================================================================================================================
show_roots_of_unity(5)

k,Real,Imaginary
0,1.0000,0.0000
1,0.3090,0.9511
2,-0.8090,0.5878
3,-0.8090,-0.5878
4,0.3090,-0.9511


In [14]:
pr( L"F_n \text{ is the matrix of eigenvectors of } P_n")
F(n) = [exp((2π*im/n)*j*k) for j=0:n-1, k=0:n-1]
display(
l_show( L"F_2 = ",        1/sympy.sqrt(2), round.(F(2),digits=0),
        L",\quad F_3 = ", 1/sympy.sqrt(3), round.(F(3),digits=2)
))
l_show(
        L"F_4 = ", 1/sympy.sqrt(4), round.(F(4),digits=0),
)

HTML{String}("<div style=\"float:center;width:100%;text-align:left;\">\n  <p style=\"height:15px;color:black;font-size:15pt;\">\$F_n \\text{ is the matrix of eigenvectors of } P_n\$</p>\n</div>")

L"$F_2 = $$\frac{\sqrt{2}}{2}$$\begin{equation}
\left(
\begin{array}{rr}
1.0+0.0\mathit{i} & 1.0+0.0\mathit{i} \\
1.0+0.0\mathit{i} & -1.0+0.0\mathit{i} \\
\end{array}
\right)
\end{equation}
$$,\quad F_3 = $$\frac{\sqrt{3}}{3}$$\begin{equation}
\left(
\begin{array}{rrr}
1.0+0.0\mathit{i} & 1.0+0.0\mathit{i} & 1.0+0.0\mathit{i} \\
1.0+0.0\mathit{i} & -0.5+0.87\mathit{i} & -0.5-0.87\mathit{i} \\
1.0+0.0\mathit{i} & -0.5-0.87\mathit{i} & -0.5+0.87\mathit{i} \\
\end{array}
\right)
\end{equation}
$"

L"$F_4 = $$\frac{1}{2}$$\begin{equation}
\left(
\begin{array}{rrrr}
1.0+0.0\mathit{i} & 1.0+0.0\mathit{i} & 1.0+0.0\mathit{i} & 1.0+0.0\mathit{i} \\
1.0+0.0\mathit{i} & \mathit{i} & -1.0+0.0\mathit{i} & -\mathit{i} \\
1.0+0.0\mathit{i} & -1.0+0.0\mathit{i} & 1.0+0.0\mathit{i} & -1.0+0.0\mathit{i} \\
1.0+0.0\mathit{i} & -\mathit{i} & -1.0+0.0\mathit{i} & \mathit{i} \\
\end{array}
\right)
\end{equation}
$"

In [31]:
# Check the eigenvalues and eigenvectors by
#   comparing the Rayleigh Coefficients λ = xᵗ F x to the roots of unity
N  = 5
Fₙ = F(N)
Pₙ = circular_shift_matrix(N)
rayleighₙ = diag(Fₙ'*Pₙ*Fₙ / N)
rootsₙ = [exp(2im * π * k / N) for k in 0:N-1]
@show  rayleighₙ ≈ rootsₙ;

rayleighₙ ≈ rootsₙ = true


#### Eigenvalues and Eigenvectors of the Circulant Matrix $C$ of Size $n\times n$

Since $\;\; C = c_0 I + c_1 P + c_2 P^2 + \dots + c_{n-1}P^{n-1}$, we see that $C$ has the same eigenvectors as $P$.

In particular, for the eigenpair $(\omega^k, x_k)$ of $P$, we have<br>
$\qquad C x_k = \left( c_0 + c_1 \omega^k + c_2 \omega^{2k} + \dots + \ c_{n-1} \omega^{(n-1)k} \right) x_k,\;\;$ i.e.,<br><br>
$\qquad$ are given by the discrete Fourier Transform of the $c$ vector.

In [16]:
C

5×5 Matrix{Sym{PyCall.PyObject}}:
 c_0  c_1  c_2  c_3  c_4
 c_4  c_0  c_1  c_2  c_3
 c_3  c_4  c_0  c_1  c_2
 c_2  c_3  c_4  c_0  c_1
 c_1  c_2  c_3  c_4  c_0

# 3. Circular Shift Matrices and Circulant Matrices

[<strong>https://web.mit.edu/18.06/www/Spring17/Circulant-Matrices.pdf</strong>](https://web.mit.edu/18.06/www/Spring17/Circulant-Matrices.pdf)

In this lecture, I want to introduce you to a new type of matrix: circulant matrices. Like Hermitian
matrices, they have orthonormal eigenvectors, but unlike Hermitian matrices we know exactly what their
eigenvectors are! Moreover, their eigenvectors are closely related to the famous Fourier transform and Fourier
series. Even more importantly, it turns out that circulant matrices and the eigenvectors lend themselves to
incredibly efficient algorithms called FFTs, that play a central role in much of computational science and
engineering

IMPORTANT: $n$ independant values, not $n^2$.

## X.Y Remarks about Linear and Cyclic Convolution

**Key Contrasts:**
* **Matrix structure:** Linear convolution uses a Toeplitz matrix, while circular convolution uses a circulant matrix.
* **Length of result:** Linear convolution produces a longer output, while circular convolution maintains the input length.
* **Periodicity:** Circular convolution assumes the input signals are periodic, while linear convolution does not.
* **Eigenstructure:** Circulant matrices (used in circular convolution) have eigenvectors that are the Fourier basis vectors,<br>
simplifying computations in the frequency domain.

**Important Remark:** In practice, **linear convolution can be computed using circular convolution**<br>
$\qquad$ by zero-padding the input vectors to length $M+N-1$ before performing the circular convolution.<br>
$\qquad$ This relationship is crucial for efficient implementation of linear convolution using Fast Fourier Transform (FFT) algorithms.

# Example Stiffness Matrix

<div style="float:left;width:40%;">
Consider the following system consisting of<br>
$\qquad n$ masses, each of mass $m$<br>
$\qquad n$ springs, each with spring constant $k$<br>
where the masses are arranged in a circle, with each mass connected to its two neighbors
</div>
<div style="float:left;padding-left:1cm;">
<img src="Figs/CirculantMatrix_MassSpringSystem.svg" width=200>
</div>

In [17]:
function circular_spring_system(n::Int; m=1, k=1)
    # Obtain the stiffness matrix
    K = Matrix(Tridiagonal(fill(-k, n-1), fill(2k, n), fill(-k, n-1)))
    K[1, n] = K[n, 1] = -k  # Connect the ends

    K
end
K = circular_spring_system(7)
l_show("Stiffness Matrix ", L"\quad K =", K)

L"$\text{Stiffness Matrix }$$\quad K =$$\begin{equation}
\left(
\begin{array}{rrrrrrr}
2 & -1 & 0 & 0 & 0 & 0 & -1 \\
-1 & 2 & -1 & 0 & 0 & 0 & 0 \\
0 & -1 & 2 & -1 & 0 & 0 & 0 \\
0 & 0 & -1 & 2 & -1 & 0 & 0 \\
0 & 0 & 0 & -1 & 2 & -1 & 0 \\
0 & 0 & 0 & 0 & -1 & 2 & -1 \\
-1 & 0 & 0 & 0 & 0 & -1 & 2 \\
\end{array}
\right)
\end{equation}
$"

This matrix has a very special pattern: every row is the same as the previous row, just shifted to the
right by 1 (wrapping around “cyclically” at the edges). That is, each row is a circular shift of the first row.
This is called a circulant matrix.

$\qquad C_n = \begin{pmatrix} c_0     &     c_1 &     c_2 &     \dots & c_{n-1} \\
                       c_{n-1} &     c_0 &     c_1 &     \dots & c_{n-2} \\
                       c_{n-2} & c_{n-1} &     c_0 &     \dots & c_{n-3} \\
                       \dots   &  \dots  & \dots   &     \dots &  \dots  \\
                       c_1     &     c_2 &     c_3 &     \dots & c_0
\end{pmatrix}$
